In [ ]:
import pandas as pd
import numpy as np
import time
import datetime
import os
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler


# Utils

In [ ]:
def split_symbol(symbol):
    return symbol.split('.')[0]

In [ ]:
def mapper(symb, mapper):
    try:
        return mapper.get(symb)
    except:
        return 'Empty'

In [ ]:
def infer_rating(df, qnt_col = 'SHARESQTY', price_col= 'SHAREPRICE', rating_col = 'RATING'):

    order_prices = df[qnt_col] * df[price_col]
    scaler = MinMaxScaler(feature_range=(1,5))
    scaled_price = scaler.fit_transform(order_prices.values.reshape(-1, 1))
    df[rating_col] = np.clip(scaled_price, 1, 5)
    return df

In [ ]:
def get_max_values(df, col_name = 'RATING', round_method = 'round'):
    max_rating_row = df.loc[df[col_name].idxmax()]
    if round_method == 'round':
        max_rating_row[col_name] = max_rating_row[col_name].round(0)
        
    elif round_method == 'ceil':
        max_rating_row[col_name] = np.ceil(max_rating_row[col_name].array)
    
    return max_rating_row

In [ ]:
def to_timestamp(date):
    return datetime.datetime.timestamp(date)

# Data reading


In [ ]:
data_path = r"D:\dev work\recommender systems\Atrad_CARS\data\portfolios_v2\raw"

portfolios = pd.DataFrame()
for fname in os.listdir(data_path):
    
    broker_df_ = pd.read_csv(os.path.join(data_path,fname), sep = '|')
    print("--- reading : {}".format(fname))
    
    portfolios = pd.concat([portfolios, broker_df_], ignore_index = True)

C:\Users\naradaw\AppData\Local\Temp\ipykernel_25200\480125734.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  broker_df_ = pd.read_csv(os.path.join(data_path,fname), sep = '|')


--- reading : Bartleet.txt


C:\Users\naradaw\AppData\Local\Temp\ipykernel_25200\480125734.py:6: DtypeWarning: Columns (2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  broker_df_ = pd.read_csv(os.path.join(data_path,fname), sep = '|')


--- reading : CAS.txt
--- reading : FCE.txt
--- reading : NLE.txt
--- reading : RPS.txt


In [ ]:
stock_info = pd.read_excel('../../data/stock_data.xlsx')
stock_info = stock_info.drop(['Unnamed: 0'],axis = 1)
stock_info.shape

(282, 4)

# Preprocessing

In [ ]:
portfolios.shape

(3728894, 8)

In [ ]:
portfolios.head()

,CDSACCNO,STOCKCODE,REFERANCE,TRAN_TYPE,SHARESQTY,SHAREPRICE,TRADE_DATE,TRADE_TIME
0,BMS-731900310-VN/00,AGPL.N0000,2024153263,B,125,7.50,5/13/2024,5/13/2024 12:25:45.000000 PM
1,BMS-800262640-VN/00,RIL.N0000,2024153264,S,-100,8.50,5/13/2024,5/13/2024 12:25:48.000000 PM
2,BMS-722151828-VN/00,PACK.N0000,2024153265,B,5000,15.00,5/13/2024,5/13/2024 12:26:03.000000 PM
3,BMS-42281-LI/00,PACK.N0000,2024153266,B,300,15.00,5/13/2024,5/13/2024 12:26:03.000000 PM
4,BMS-478-LC/00,HNB.N0000,2024153267,B,350,202.25,5/13/2024,5/13/2024 12:26:42.000000 PM


In [ ]:
portfolios['TRADE_TIME'] = pd.to_datetime(portfolios['TRADE_TIME'])
portfolios['TRADE_DATE'] = pd.to_datetime(portfolios['TRADE_DATE'])

In [ ]:
portfolios.dtypes

CDSACCNO              object
STOCKCODE             object
REFERANCE             object
TRAN_TYPE             object
SHARESQTY              int64
SHAREPRICE           float64
TRADE_DATE    datetime64[ns]
TRADE_TIME    datetime64[ns]
dtype: object

In [ ]:
stock_info = stock_info.dropna()

In [ ]:
stock_info.head()

,symbol,name,buisnesssummary,gics_code
0,HBS,hSenid Business Solutions PLC,An indigenous multinational catering towards m...,45103010 - Application Software
1,TYRE,KELANI TYRES PLC,Manufacturing tyres and tubes and marketing lo...,Automobiles & Components
2,ABL,AMANA BANK PLC,unknown,Banks
3,DFCC,DFCC BANK PLC,The principal activities of DFCC Bank include ...,Banks
4,COMB,COMMERCIAL BANK OF CEYLON PLC,Commercial Banking,Banks


In [ ]:
unique_symbols = set(stock_info.symbol.unique())
len(unique_symbols)

280

In [ ]:
# Only selecting Buy orders
portfolios_df = portfolios.copy()
portfolios_df = portfolios_df.loc[portfolios_df.TRAN_TYPE == 'B']
portfolios_df.shape

In [ ]:
portfolios_df.head()

,CDSACCNO,STOCKCODE,REFERANCE,TRAN_TYPE,SHARESQTY,SHAREPRICE,TRADE_DATE,TRADE_TIME
0,BMS-731900310-VN/00,AGPL.N0000,2024153263,B,125,7.50,2024-05-13,2024-05-13 12:25:45
2,BMS-722151828-VN/00,PACK.N0000,2024153265,B,5000,15.00,2024-05-13,2024-05-13 12:26:03
3,BMS-42281-LI/00,PACK.N0000,2024153266,B,300,15.00,2024-05-13,2024-05-13 12:26:03
4,BMS-478-LC/00,HNB.N0000,2024153267,B,350,202.25,2024-05-13,2024-05-13 12:26:42
11,BMS-478-LC/00,HNB.N0000,2024153274,B,109,202.25,2024-05-13,2024-05-13 12:29:02


In [ ]:
portfolios_df_fil_1 = portfolios_df.groupby(by = 'CDSACCNO').filter(lambda x: x['STOCKCODE'].nunique() > 10)
portfolios_df_fil_1['UNIX_TS'] = portfolios_df_fil_1['TRADE_DATE'].apply(lambda x: to_timestamp(x))
portfolios_df_fil_1.head()

,CDSACCNO,STOCKCODE,REFERANCE,TRAN_TYPE,SHARESQTY,SHAREPRICE,TRADE_DATE,TRADE_TIME,UNIX_TS
0,BMS-731900310-VN/00,AGPL.N0000,2024153263,B,125,7.50,2024-05-13,2024-05-13 12:25:45,1.715539e+09
2,BMS-722151828-VN/00,PACK.N0000,2024153265,B,5000,15.00,2024-05-13,2024-05-13 12:26:03,1.715539e+09
3,BMS-42281-LI/00,PACK.N0000,2024153266,B,300,15.00,2024-05-13,2024-05-13 12:26:03,1.715539e+09
4,BMS-478-LC/00,HNB.N0000,2024153267,B,350,202.25,2024-05-13,2024-05-13 12:26:42,1.715539e+09
11,BMS-478-LC/00,HNB.N0000,2024153274,B,109,202.25,2024-05-13,2024-05-13 12:29:02,1.715539e+09


In [ ]:
portfolios_df_fil_1.CDSACCNO.nunique(), portfolios_df_fil_1.STOCKCODE.nunique()

(5906, 346)

In [ ]:
prev_symbols = set(list(portfolios_df_fil_1.STOCKCODE.unique()))
portfolios_df_fil_1.shape

(1674385, 9)

In [ ]:
portfolios_df_fil_1['STOCKCODE'] = portfolios_df_fil_1.STOCKCODE.apply(lambda x : split_symbol(x))

In [73]:
portfolios_df_fil_1['STOCKCODE'] = portfolios_df_fil_1.STOCKCODE.apply(lambda x : split_symbol(x))

In [74]:
unique_port_symbols = set(portfolios_df_fil_1.STOCKCODE.unique())
len(unique_port_symbols)

288

In [75]:
#remove symbols that do not have their details in the stock data dataset

to_remove = list(unique_port_symbols - unique_symbols)
to_remove

['YORK',
 'GSF',
 'WIND',
 'AGPL',
 'CITW',
 'WATA',
 'LGIL',
 'CBNK',
 'CLC',
 'CALI',
 'PDL',
 'UBF',
 'SFL']

In [76]:
portfolios_df_fil_1 = portfolios_df_fil_1[~portfolios_df_fil_1.STOCKCODE.isin(to_remove)]

In [77]:
symb_to_name = dict(zip(stock_info.symbol,stock_info.name))
symb_to_gics = dict(zip(stock_info.symbol, stock_info.gics_code))

In [78]:
portfolios_df_fil_1['STOCKNAME'] = portfolios_df_fil_1.STOCKCODE.apply(lambda x: mapper(x,symb_to_name))
portfolios_df_fil_1['GICS'] = portfolios_df_fil_1.STOCKCODE.apply(lambda x: mapper(x,symb_to_gics))

In [79]:
# portfolios_df_fil_1[portfolios_df_fil_1['STOCKNAME'] == np.nan]

In [80]:
portfolios_df_fil_1.CDSACCNO.nunique(), portfolios_df_fil_1.STOCKCODE.nunique()

(5906, 275)

In [81]:


# portfolios_df_fil_3 = filter_portfolios(portfolios_df_fil_2)
portfolios_df_fil_3 = portfolios_df_fil_1.groupby('CDSACCNO', group_keys= False).apply(lambda x: infer_rating(x)).groupby(['CDSACCNO','STOCKCODE'], group_keys= False).apply(lambda x: get_max_values(x)).reset_index(drop =True).sort_values('RATING', ascending= False)

C:\Users\naradaw\AppData\Local\Temp\ipykernel_25200\184762752.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  portfolios_df_fil_3 = portfolios_df_fil_1.groupby('CDSACCNO', group_keys= False).apply(lambda x: infer_rating(x)).groupby(['CDSACCNO','STOCKCODE'], group_keys= False).apply(lambda x: get_max_values(x)).reset_index(drop =True).sort_values('RATING', ascending= False)
C:\Users\naradaw\AppData\Local\Temp\ipykernel_25200\184762752.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings

In [82]:
portfolios_df_fil_3.CDSACCNO.nunique(), portfolios_df_fil_3.STOCKCODE.nunique()

(5906, 275)

In [83]:
portfolios_df_fil_3.head(2)

,CDSACCNO,STOCKCODE,REFERANCE,TRAN_TYPE,SHARESQTY,SHAREPRICE,TRADE_DATE,TRADE_TIME,UNIX_TS,STOCKNAME,GICS,RATING
24126,BMS-48991-LI/00,LWL,24595,B,1365,49.0,2023-01-11,NaT,1.673375e+09,LANKA WALLTILE PLC,Capital Goods,5.0
140760,HDF-743612299-VN/00,SEMB,2023178157,B,10000,0.7,2023-07-28,2023-07-28 01:15:30,1.690483e+09,S M B LEASING PLC,Diversified Financials,5.0


In [84]:
portfolios_df_fil_3.GICS.nunique()

33

In [85]:
portfolios_df_fil_4 = portfolios_df_fil_3[['CDSACCNO','STOCKCODE','UNIX_TS','RATING','GICS','STOCKNAME']] #,'GICS','STOCKNAME'

In [86]:
portfolios_df_fil_4

,CDSACCNO,STOCKCODE,UNIX_TS,RATING,GICS,STOCKNAME
24126,BMS-48991-LI/00,LWL,1.673375e+09,5.0,Capital Goods,LANKA WALLTILE PLC
140760,HDF-743612299-VN/00,SEMB,1.690483e+09,5.0,Diversified Financials,S M B LEASING PLC
91055,CAS-2556610-NP/00,HASU,1.642012e+09,5.0,Insurance,HNB ASSURANCE PLC
109130,COM-69412-LI/00,LOFC,1.664822e+09,5.0,Diversified Financials,LOLC FINANCE PLC
140787,HDF-74565-LI/00,FCT,1.690742e+09,5.0,Investment Banking & Brokerage,First Capital Treasuries PLC
...,...,...,...,...,...,...
42772,BMS-67444-LI/00,KVAL,1.652035e+09,1.0,Food Beverage & Tobacco,KELANI VALLEY PLANTATIONS PLC
103268,CAS-90528-LI/00,FCT,1.712687e+09,1.0,Investment Banking & Brokerage,First Capital Treasuries PLC
103265,CAS-90528-LI/00,CTBL,1.713465e+09,1.0,Retailing,CEYLON TEA BROKERS PLC
103263,CAS-90528-LI/00,CALT,1.713465e+09,1.0,Investment Banking & Brokerage,CAPITAL ALLIANCE PLC


In [87]:
train_ = portfolios_df_fil_4.groupby('CDSACCNO', group_keys=False).apply(lambda x: x.sample(frac=0.8))
train_

C:\Users\naradaw\AppData\Local\Temp\ipykernel_25200\1470628542.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_ = portfolios_df_fil_4.groupby('CDSACCNO', group_keys=False).apply(lambda x: x.sample(frac=0.8))


,CDSACCNO,STOCKCODE,UNIX_TS,RATING,GICS,STOCKNAME
0,BMS-10073-LC/00,AAIC,1.676313e+09,1.0,Insurance,SOFTLOGIC LIFE INSURANCE PLC
9,BMS-10073-LC/00,SHL,1.642617e+09,1.0,Capital Goods,SOFTLOGIC HOLDINGS PLC
5,BMS-10073-LC/00,HDFC,1.644777e+09,1.0,Banks,HOUSING DEVELOPMENT FINANCE CORPORATION BANK O...
6,BMS-10073-LC/00,LIOC,1.641494e+09,5.0,Energy,LANKA IOC PLC
3,BMS-10073-LC/00,CONN,1.642531e+09,1.0,Consumer Services,HAYLEYS LEISURE PLC
...,...,...,...,...,...,...
157844,SCB-7874-LC/00,JETS,1.694371e+09,1.0,Consumer Services,JETWING SYMPHONY PLC
157839,SCB-7874-LC/00,CFIN,1.693852e+09,3.0,Diversified Financials,CENTRAL FINANCE COMPANY PLC
157853,SCB-7874-LC/00,TJL,1.704998e+09,2.0,Consumer Durables & Apparel,TEEJAY LANKA PLC
157841,SCB-7874-LC/00,FCT,1.666031e+09,1.0,Investment Banking & Brokerage,First Capital Treasuries PLC


In [88]:
test_ = portfolios_df_fil_4[~portfolios_df_fil_4.index.isin(train_.index)]
test_

,CDSACCNO,STOCKCODE,UNIX_TS,RATING,GICS,STOCKNAME
140787,HDF-74565-LI/00,FCT,1.690742e+09,5.0,Investment Banking & Brokerage,First Capital Treasuries PLC
63605,BMS-800262640-VN/00,VONE,1.713119e+09,5.0,Utilities,VALLIBEL ONE PLC
109170,COM-69742-LC/00,RCL,1.662662e+09,5.0,Capital Goods,ROYAL CERAMICS LANKA PLC
77248,BMS-861802000-VN/00,TPL,1.681151e+09,5.0,Food Beverage & Tobacco,TALAWAKELLE TEA ESTATES PLC
140716,HDF-743463188-VN/00,CTBL,1.690137e+09,5.0,Retailing,CEYLON TEA BROKERS PLC
...,...,...,...,...,...,...
103245,CAS-90482-LI/00,ASPH,1.714329e+09,1.0,Materials,INDUSTRIAL ASPHALTS (CEYLON) PLC
103260,CAS-90528-LI/00,ABL,1.713983e+09,1.0,Banks,AMANA BANK PLC
103277,CAS-90528-LI/00,LOFC,1.713465e+09,1.0,Diversified Financials,LOLC FINANCE PLC
42769,BMS-67444-LI/00,HNBF,1.684262e+09,1.0,Banks Finance & Insurance,HNB FINANCE PLC


In [89]:
portfolios_df_fil_4.shape[0] == train_.shape[0] + test_.shape[0]

True

In [90]:
train_.CDSACCNO.nunique(), test_.CDSACCNO.nunique()

(5906, 5906)

In [91]:
data_dict = portfolios_df_fil_4.to_dict(orient='list')
dataset = tf.data.Dataset.from_tensor_slices(data_dict)
dataset.save("../../data/portfolios_v2/portfolios")

In [92]:
data_dict = train_.to_dict(orient='list')
train_dataset = tf.data.Dataset.from_tensor_slices(data_dict)
train_dataset.save("../../data/portfolios_v2/retriver_train")

In [93]:
data_dict = test_.to_dict(orient='list')
test_dataset = tf.data.Dataset.from_tensor_slices(data_dict)
test_dataset.save("../../data/portfolios_v2/retriver_test")

In [94]:
len(dataset), len(train_dataset), len(test_dataset)

(157854, 126347, 31507)

In [95]:
train_hoo, test_hoo = pd.DataFrame(), pd.DataFrame()
for name, group in portfolios_df_fil_4.groupby('CDSACCNO'):
  train_hoo = pd.concat([train_hoo, group.iloc[0:-1]], ignore_index=True)  # Take first row as test
  test_hoo = pd.concat([test_hoo, group.iloc[-1:]], ignore_index=True)  # Rest for train

In [96]:
train_hoo.CDSACCNO.nunique(), test_hoo.CDSACCNO.nunique()

(5906, 5906)

In [97]:
train_hoo

,CDSACCNO,STOCKCODE,UNIX_TS,RATING,GICS,STOCKNAME
0,BMS-10073-LC/00,LIOC,1.641494e+09,5.0,Energy,LANKA IOC PLC
1,BMS-10073-LC/00,LOLC,1.646591e+09,3.0,Diversified Financials,L O L C HOLDINGS PLC
2,BMS-10073-LC/00,CONN,1.642531e+09,1.0,Consumer Services,HAYLEYS LEISURE PLC
3,BMS-10073-LC/00,CSF,1.644863e+09,1.0,Diversified Financials,NATION LANKA FINANCE PLC
4,BMS-10073-LC/00,AMF,1.643827e+09,1.0,Diversified Financials,ASSOCIATED MOTOR FINANCE COMPANY PLC
...,...,...,...,...,...,...
151943,SCB-7874-LC/00,JKH,1.698863e+09,1.0,Capital Goods,JOHN KEELLS HOLDINGS PLC
151944,SCB-7874-LC/00,DIMO,1.686767e+09,1.0,Retailing,DIESEL & MOTOR ENGINEERING PLC
151945,SCB-7874-LC/00,FCT,1.666031e+09,1.0,Investment Banking & Brokerage,First Capital Treasuries PLC
151946,SCB-7874-LC/00,HNB,1.687459e+09,1.0,Banks,HATTON NATIONAL BANK PLC


In [98]:
test_hoo

,CDSACCNO,STOCKCODE,UNIX_TS,RATING,GICS,STOCKNAME
0,BMS-10073-LC/00,AAIC,1.676313e+09,1.0,Insurance,SOFTLOGIC LIFE INSURANCE PLC
1,BMS-10078-LC/00,CFVF,1.691606e+09,1.0,Diversified Financials,FIRST CAPITAL HOLDINGS PLC
2,BMS-10089-LC/00,SPEN,1.644777e+09,1.0,Capital Goods,AITKEN SPENCE PLC
3,BMS-10284-LC/00,VPEL,1.686681e+09,1.0,Power and Energy,VALLIBEL POWER ERATHNA PLC
4,BMS-10463-LC/00,ASIY,1.646851e+09,1.0,Diversified Financials,ASIA SIYAKA COMMODITIES LIMITED
...,...,...,...,...,...,...
5901,SBL-762412608-VN/00,ELPL,1.683743e+09,1.0,Food Beverage & Tobacco,ELPITIYA PLANTATIONS PLC
5902,SCB-11576-LC/00,KHL,1.662662e+09,1.0,Consumer Services,JOHN KEELLS HOTELS PLC
5903,SCB-11577-LC/00,PLC,1.692124e+09,1.0,Diversified Financials,PEOPLE'S LEASING & FINANCE PLC
5904,SCB-1522-LC/00,TJL,1.646937e+09,1.0,Consumer Durables & Apparel,TEEJAY LANKA PLC


In [99]:
data_dict = train_hoo.to_dict(orient='list')
train_hoo_dataset = tf.data.Dataset.from_tensor_slices(data_dict)
train_hoo_dataset.save("../../data/portfolios_v2/retriver_hoo_train")

In [100]:
data_dict = test_hoo.to_dict(orient='list')
test_hoo_dataset = tf.data.Dataset.from_tensor_slices(data_dict)
test_hoo_dataset.save("../../data/portfolios_v2/retriver_hoo_test")

In [101]:
len(dataset), len(train_hoo_dataset), len(test_hoo_dataset)

(157854, 151948, 5906)

In [102]:
# tf.random.set_seed(42)
# shuffled = dataset.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

# train = shuffled.take(int(len(dataset)* 0.8))
# test = shuffled.skip(int(len(dataset)* 0.8)).take(int(len(dataset)* 0.2))

In [103]:
# train.save("../../data/portfolios_v2/retriver_train")
# test.save("../../data/portfolios_v2/retriver_test")

In [104]:
# new_dataset = tf.data.Dataset.load("../../data/portfolios_v2/portfolios_tfds")

# Work Here

In [105]:
import array
import collections

from typing import Dict, List, Optional, Text, Tuple

def _create_feature_dict() -> Dict[Text, List[tf.Tensor]]:
  return {"STOCKCODE": [], "RATING": [], "GICS": [], "STOCKNAME": [], "UNIX_TS": []}

def _sample_list(
    feature_lists: Dict[Text, List[tf.Tensor]],
    num_examples_per_list: int,
    random_state: Optional[np.random.RandomState] = None,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Function for sampling a list example from given feature lists."""
  if random_state is None:
    random_state = np.random.RandomState()

  sampled_indices = random_state.choice(
      range(len(feature_lists["STOCKCODE"])),
      size=num_examples_per_list,
      replace=False,
  )
  sampled_STOCKCODE = [
      feature_lists["STOCKCODE"][idx] for idx in sampled_indices
  ]
  sampled_RATING = [
      feature_lists["RATING"][idx]
      for idx in sampled_indices
  ]
  sampled_GICS = [
      feature_lists["GICS"][idx] for idx in sampled_indices
  ]
  sampled_STOCKNAME = [
      feature_lists["STOCKNAME"][idx]
      for idx in sampled_indices
  ]
  sampled_UNIX_TS = [
      feature_lists["UNIX_TS"][idx] for idx in sampled_indices
  ]

  return (
      tf.stack(sampled_STOCKCODE, 0),
      tf.stack(sampled_RATING, 0),
      tf.stack(sampled_GICS, 0),
      tf.stack(sampled_STOCKNAME, 0),
      tf.stack(sampled_UNIX_TS, 0)
  )


def sample_listwise(
    rating_dataset: tf.data.Dataset,
    num_list_per_user: int = 10,
    num_examples_per_list: int = 10,
    seed: Optional[int] = None,
) -> tf.data.Dataset:
  
  random_state = np.random.RandomState(seed)

  example_lists_by_user = collections.defaultdict(_create_feature_dict)

  movie_title_vocab = set()
  for example in rating_dataset:
    user_id = example["CDSACCNO"].numpy()
    example_lists_by_user[user_id]["STOCKCODE"].append(
        example["STOCKCODE"])
    example_lists_by_user[user_id]["RATING"].append(
        example["RATING"])
    example_lists_by_user[user_id]["GICS"].append(
        example["GICS"])
    example_lists_by_user[user_id]["STOCKNAME"].append(
        example["STOCKNAME"])
    example_lists_by_user[user_id]["UNIX_TS"].append(
        example["UNIX_TS"])
    
    movie_title_vocab.add(example["STOCKNAME"].numpy())

    

  tensor_slices = {"CDSACCNO": [], "STOCKCODE": [], "RATING": [], "GICS": [], "STOCKNAME": [], "UNIX_TS": []}

  for user_id, feature_lists in example_lists_by_user.items():
    for _ in range(num_list_per_user):

      # Drop the user if they don't have enough ratings.
      if len(feature_lists["STOCKNAME"]) < num_examples_per_list:
        continue

        '''sampled_STOCKCODE, 0),
      tf.stack(sampled_RATING, 0),
      tf.stack(sampled_GICS, 0),
      tf.stack(sampled_STOCKNAME, 0),
      tf.stack(sampled_UNIX_TS'''

      sampled_STOCKCODE, sampled_RATING, sampled_GICS, sampled_STOCKNAME, sampled_UNIX_TS  = _sample_list(
          feature_lists,
          num_examples_per_list,
          random_state=random_state,
      )
      tensor_slices["CDSACCNO"].append(user_id)
      tensor_slices["STOCKCODE"].append(sampled_STOCKCODE)
      tensor_slices["RATING"].append(sampled_RATING)
      tensor_slices["GICS"].append(sampled_GICS)
      tensor_slices["STOCKNAME"].append(sampled_STOCKNAME)
      tensor_slices["UNIX_TS"].append(sampled_UNIX_TS)

  return tf.data.Dataset.from_tensor_slices(tensor_slices)

In [106]:
# portfolios = tf.data.Dataset.load("../../data/portfolios_tfds_lists")
portfolios = dataset

In [107]:
# train_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/train_lists").cache() #data\ratings_train
# test_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/test_lists").cache()

train_ds = train
test_ds = test

NameError: name 'train' is not defined

In [ ]:
next(iter(train_ds)), len(train_ds)

({'CDSACCNO': <tf.Tensor: shape=(), dtype=string, numpy=b'HDF-733381418-VN/00'>,
  'STOCKCODE': <tf.Tensor: shape=(), dtype=string, numpy=b'LWL'>,
  'UNIX_TS': <tf.Tensor: shape=(), dtype=float32, numpy=1660501800.0>,
  'RATING': <tf.Tensor: shape=(), dtype=float32, numpy=2.0>,
  'GICS': <tf.Tensor: shape=(), dtype=string, numpy=b'Capital Goods'>,
  'STOCKNAME': <tf.Tensor: shape=(), dtype=string, numpy=b'LANKA WALLTILE PLC'>},
 126283)

In [ ]:
train_v1 = sample_listwise(
    train_ds,
    num_list_per_user=50,
    num_examples_per_list=10,
    seed=42
)

test_v1 = sample_listwise(
    test_ds,
    num_list_per_user=1,
    num_examples_per_list=10,
    seed=42
)

In [ ]:
next(iter(train_v1))

{'CDSACCNO': <tf.Tensor: shape=(), dtype=string, numpy=b'HDF-733381418-VN/00'>,
 'STOCKCODE': <tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'LIOC', b'LITE', b'MBSL', b'PLC', b'LWL', b'ALLI', b'SHL',
        b'HAYL', b'CFVF', b'KAHA'], dtype=object)>,
 'RATING': <tf.Tensor: shape=(10,), dtype=float32, numpy=array([2., 1., 1., 1., 2., 2., 1., 1., 1., 2.], dtype=float32)>,
 'GICS': <tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'Energy', b'Capital Goods', b'Diversified Financials',
        b'Diversified Financials', b'Capital Goods',
        b'Diversified Financials', b'Capital Goods', b'Capital Goods',
        b'Diversified Financials', b'Food Beverage & Tobacco'],
       dtype=object)>,
 'STOCKNAME': <tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'LANKA IOC PLC', b'LAXAPANA BATTERIES PLC',
        b'MERCHANT BANK OF SRI LANKA & FINANCE PLC',
        b"PEOPLE'S LEASING & FINANCE PLC", b'LANKA WALLTILE PLC',
        b'ALLIANCE FINANCE COMPANY PLC', b'SOFTLOGIC HOL

In [ ]:
len(train_v1)

258200

In [ ]:
train_v1.save("../../data/portfolios_v2/ranker_train")
test_v1.save("../../data/portfolios_v2/ranker_test")

In [ ]:
train_ds = tf.data.Dataset.load("D:\dev work\recommender systems\Atrad_CARS\data\portfolios_v2\retriver_train").cache()

InvalidArgumentError: NewRandomAccessFile failed to Create/Open: D:\dev workecommender systems\Atrad_CARS\data\portfolios_v2etriver_train\dataset_spec.pb : The filename, directory name, or volume label syntax is incorrect.
; no protocol option